In [1]:
### Note: This notebook runs on Tensorflow 1.14
from sklearn.datasets import fetch_california_housing
import numpy as np
from TensorTree_Model import TensorTree_Based
import tensorflow as tf
import keras 
from keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import time

def local_feature_mapping(x_train,x_val,x_test,local_dim=2,unit_norm=False):
    
    n_train_samples = x_train.shape[0]
    n_val_samples = x_val.shape[0]
    n_test_samples = x_test.shape[0]
    n_features = x_train.shape[1]

    X_transformed_train_list = []
    X_transformed_val_list = []
    X_transformed_test_list = []
    
    for d in range(local_dim-1):
        X_transformed_train_list.append(x_train**(d+1))
        X_transformed_val_list.append(x_val**(d+1))
        X_transformed_test_list.append(x_test**(d+1))
         
    X_stand_train_list = [np.ones((n_train_samples,n_features))]
    X_stand_val_list = [np.ones((n_val_samples,n_features))]
    X_stand_test_list = [np.ones((n_test_samples,n_features))]
    
    for X_trans_train,X_trans_val,X_trans_test in zip(X_transformed_train_list,X_transformed_val_list,X_transformed_test_list):
        
      X_stand_train_list.append(X_trans_train)
      X_stand_val_list.append(X_trans_val) 
      X_stand_test_list.append(X_trans_test)
        
    x_train_processed= np.transpose(X_stand_train_list,(1, 2, 0))
    x_val_processed= np.transpose(X_stand_val_list,(1, 2, 0))
    x_test_processed = np.transpose(X_stand_test_list,(1, 2, 0))

    if(unit_norm==True):
      norms_train = np.sqrt(np.sum(x_train_processed**2,axis=-1,keepdims=True))
      x_train_processed = np.divide(x_train_processed,norms_train)

      norms_val = np.sqrt(np.sum(x_val_processed**2,axis=-1,keepdims=True))
      x_val_processed = np.divide(x_val_processed,norms_val)
    
      norms_test = np.sqrt(np.sum(x_test_processed**2,axis=-1,keepdims=True))
      x_test_processed = np.divide(x_test_processed,norms_test)

    return x_train_processed, x_val_processed, x_test_processed      

    ### Data preprocessing ###
dataset = fetch_california_housing()
X_full, Y_full = dataset.data, dataset.target
x_train, x_test, y_train, y_test = train_test_split(X_full, Y_full, test_size = 0.2, random_state = 0)
x_train, x_val, y_train, y_val = x_train[:13208],x_train[13208:], y_train[:13208],y_train[13208:]

scaler_X = StandardScaler()
scaler_Y = StandardScaler()

x_train, x_val, x_test= scaler_X.fit_transform(x_train), scaler_X.transform(x_val), scaler_X.transform(x_test)
y_train, y_val, y_test = scaler_Y.fit_transform(y_train.reshape(-1,1)).ravel(), scaler_Y.transform(y_val.reshape(-1,1)).ravel(), scaler_Y.transform(y_test.reshape(-1,1)).ravel()

#%%
rank=10
local_dim=75

x_train_processed,x_val_processed,x_test_processed=local_feature_mapping(x_train,x_val,x_test,local_dim=local_dim,unit_norm=True)

print('rank: {}, local dimension:{}'.format(rank,local_dim))

model=Sequential()
model.add(TensorTree_Based(units=1,activation=None,rank=rank,local_dim=local_dim,regularizer=keras.regularizers.l2(3e-7),

                    initializer=keras.initializers.TruncatedNormal(mean=0, stddev=0.2)))

optimizer = keras.optimizers.Adam(lr=1e-3)
model.compile(optimizer, loss=tf.keras.losses.MeanSquaredError())

start = time.time()
history=model.fit(x_train_processed, y_train, epochs=100, batch_size=32,validation_data=(x_val_processed,y_val))
end = time.time()

val_loss=min(history.history['val_loss'])
print('validation loss:{}'.format(val_loss))
   
print('Training time:{}'.format(end-start))

/opt/anaconda3/envs/RL_Trade/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/anaconda3/envs/RL_Trade/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/anaconda3/envs/RL_Trade/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/anaconda3/envs/RL_Trade/lib/python3.7/site-packages/tensorf

rank: 10, local dimension:75






/opt/anaconda3/envs/RL_Trade/lib/python3.7/site-packages/ipykernel_launcher.py:49: RuntimeWarning: overflow encountered in square




Train on 13208 samples, validate on 3304 samples
Epoch 1/100
13208/13208 [==============================] - 2s 146us/step - loss: 0.5495 - val_loss: 0.3692
Epoch 2/100
13208/13208 [==============================] - 1s 83us/step - loss: 0.3099 - val_loss: 0.2650
Epoch 3/100
13208/13208 [==============================] - 1s 86us/step - loss: 0.2462 - val_loss: 0.2318
Epoch 4/100
13208/13208 [==============================] - 1s 88us/step - loss: 0.2278 - val_loss: 0.2280
Epoch 5/100
13208/13208 [==============================] - 1s 72us/step - loss: 0.2192 - val_loss: 0.2125
Epoch 6/100
13208/13208 [==============================] - 1s 71us/step - loss: 0.2114 - val_loss: 0.2195
Epoch 7/100
13208/13208 [==============================] - 1s 73us/step - loss: 0.2068 - val_loss: 0.2139
Epoch 8/100
13208/13208 [==============================] - 1s 73us/step - loss: 0.2053 - val_loss: 0.2132
Epoch 9/100
13208/13208 [==============================] - 1s 73us/step - loss: 0.2028 - val_loss: 0

13208/13208 [==============================] - 1s 69us/step - loss: 0.1403 - val_loss: 0.2816
Epoch 74/100
13208/13208 [==============================] - 1s 78us/step - loss: 0.1406 - val_loss: 0.2916
Epoch 75/100
13208/13208 [==============================] - 1s 82us/step - loss: 0.1411 - val_loss: 0.2841
Epoch 76/100
13208/13208 [==============================] - 1s 82us/step - loss: 0.1391 - val_loss: 0.2993
Epoch 77/100
13208/13208 [==============================] - 1s 78us/step - loss: 0.1387 - val_loss: 0.3016
Epoch 78/100
13208/13208 [==============================] - 1s 70us/step - loss: 0.1369 - val_loss: 0.2975
Epoch 79/100
13208/13208 [==============================] - 1s 74us/step - loss: 0.1370 - val_loss: 0.2937
Epoch 80/100
13208/13208 [==============================] - 1s 72us/step - loss: 0.1364 - val_loss: 0.3075
Epoch 81/100
13208/13208 [==============================] - 1s 73us/step - loss: 0.1352 - val_loss: 0.2858
Epoch 82/100
13208/13208 [========================